In [1]:
import pandas as pd
import numpy as np

PATH = '/home/kai/data/kaggle/homecredit/'
train = pd.read_csv(PATH + 'application_train.csv')
print('train')
test = pd.read_csv(PATH + 'application_test.csv')
print('test')
# bureau = pd.read_csv(PATH + 'bureau.csv')
# print('bureau')
# previous_application = pd.read_csv(PATH + 'previous_application.csv')
# print('previous')
# bureau_balance = pd.read_csv(PATH + 'bureau_balance.csv')
# print('bureau_balance')
# installments_payments = pd.read_csv(PATH + 'installments_payments.csv')
# print('installments_payments done')
credit_card_balance = pd.read_csv(PATH + 'credit_card_balance.csv')
print('credit_card_balance')
# POS_CASH_balance = pd.read_csv(PATH + 'POS_CASH_balance.csv')
# print('POS_CASH_balance')
# credit_card_balance = pd.read_csv(PATH + 'credit_card_balance.csv')
# print('credit_card_balance')
# previous_application = pd.read_csv(PATH + 'previous_application.csv')
# print('previous_application')

train
test
credit_card_balance


In [2]:
credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [3]:
credit_card_balance[credit_card_balance['AMT_DRAWINGS_CURRENT'] ==\
   credit_card_balance['AMT_DRAWINGS_ATM_CURRENT']+
   credit_card_balance['AMT_DRAWINGS_POS_CURRENT']+
   credit_card_balance['AMT_DRAWINGS_OTHER_CURRENT']].shape
3078191+749816

3828007

In [4]:
credit_card_balance.shape

(3840312, 23)

In [5]:
credit_card_balance[credit_card_balance['CNT_DRAWINGS_CURRENT'] ==\
   credit_card_balance['CNT_DRAWINGS_OTHER_CURRENT']+
   credit_card_balance['CNT_DRAWINGS_POS_CURRENT']+
   credit_card_balance['CNT_DRAWINGS_ATM_CURRENT']].shape

(3090496, 23)

In [6]:
set(credit_card_balance['NAME_CONTRACT_STATUS'])

{'Active',
 'Approved',
 'Completed',
 'Demand',
 'Refused',
 'Sent proposal',
 'Signed'}

# Substraction, ratio, and score_dpd

In [7]:
import pandas as pd
import os
import gc
import numpy as np

def ratio_name(numerator, denominator): return numerator + '_' + denominator + '_ratio'

def ratio(df, numerator, denominator):
    return df[numerator] / df[denominator]

def substraction_name(col1, col2): return col1 + '_minus_' + col2

def substraction(df, col1, col2):
    return df[col1] - df[col2]

In [8]:
# Substraction
credit_card_balance[substraction_name('AMT_CREDIT_LIMIT_ACTUAL', 'AMT_BALANCE')] = \
substraction(credit_card_balance, 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_BALANCE')

In [9]:
# Ratio
top = ['AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',\
      'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',\
      'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_CURRENT']

bot = ['AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_CURRENT','AMT_DRAWINGS_CURRENT', \
       'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_CURRENT',\
       'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_CURRENT']

In [10]:
# Ratio
for i in range(len(top)):
    print(top[i],bot[i])
    credit_card_balance[ratio_name(top[i], bot[i])] = \
    ratio(credit_card_balance, top[i], bot[i])

AMT_DRAWINGS_ATM_CURRENT AMT_DRAWINGS_CURRENT
AMT_DRAWINGS_POS_CURRENT AMT_DRAWINGS_CURRENT
AMT_DRAWINGS_OTHER_CURRENT AMT_DRAWINGS_CURRENT
CNT_DRAWINGS_ATM_CURRENT CNT_DRAWINGS_CURRENT
CNT_DRAWINGS_OTHER_CURRENT CNT_DRAWINGS_CURRENT
CNT_DRAWINGS_POS_CURRENT CNT_DRAWINGS_CURRENT
AMT_DRAWINGS_ATM_CURRENT CNT_DRAWINGS_ATM_CURRENT
AMT_DRAWINGS_POS_CURRENT CNT_DRAWINGS_POS_CURRENT
AMT_DRAWINGS_OTHER_CURRENT CNT_DRAWINGS_OTHER_CURRENT
AMT_DRAWINGS_CURRENT CNT_DRAWINGS_CURRENT


In [20]:
credit_card_balance['score_DPD'] = 0
credit_card_balance = credit_card_balance.sort_values('MONTHS_BALANCE')
credit_card_balance['score_DPD'] = (credit_card_balance['SK_DPD']**3 + 5* credit_card_balance['SK_DPD_DEF']**3 + 1) * (1/(1 + abs(credit_card_balance['MONTHS_BALANCE']) ))

In [12]:
credit[['score_DPD','SK_DPD','SK_DPD_DEF', 'MONTHS_BALANCE' ]]

,score_DPD,SK_DPD,SK_DPD_DEF,MONTHS_BALANCE
24268,0.010309,0,0,-96
841327,0.010309,0,0,-96
2725728,0.010309,0,0,-96
2725726,0.010309,0,0,-96
841332,0.010309,0,0,-96
2725714,0.010309,0,0,-96
2725711,0.010309,0,0,-96
841338,0.072165,1,1,-96
2725708,0.010309,0,0,-96
841340,0.010309,0,0,-96


In [13]:
credit_card_balance.columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE',
       'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT',
       'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT',
       'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM',
       'NAME_CONTRACT_STATUS', 'SK_DPD', 'SK_DPD_DEF',
       'AMT_CREDIT_LIMIT_ACTUAL_minus_AMT_BALANCE',
       'AMT_DRAWINGS_ATM_CURRENT_AMT_DRAWINGS_CURRENT_ratio',
       'AMT_DRAWINGS_POS_CURRENT_AMT_DRAWINGS_CURRENT_ratio',
       'AMT_DRAWINGS_OTHER_CURRENT_AMT_DRAWINGS_CURRENT_ratio',
       'CNT_DRAWINGS_ATM_CURRENT_CNT_DRAWINGS_CURRENT_ratio',
       'CNT_DRAWINGS_OTHER_CURRENT_CNT_DRAWINGS_CURRENT_ratio',
       'CNT_DRAWINGS_POS_CURREN

In [14]:
col = [i for i in list(credit_card_balance.columns.values) if credit_card_balance[i].dtype!= 'object']
print(col)

['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY', 'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM', 'SK_DPD', 'SK_DPD_DEF', 'AMT_CREDIT_LIMIT_ACTUAL_minus_AMT_BALANCE', 'AMT_DRAWINGS_ATM_CURRENT_AMT_DRAWINGS_CURRENT_ratio', 'AMT_DRAWINGS_POS_CURRENT_AMT_DRAWINGS_CURRENT_ratio', 'AMT_DRAWINGS_OTHER_CURRENT_AMT_DRAWINGS_CURRENT_ratio', 'CNT_DRAWINGS_ATM_CURRENT_CNT_DRAWINGS_CURRENT_ratio', 'CNT_DRAWINGS_OTHER_CURRENT_CNT_DRAWINGS_CURRENT_ratio', 'CNT_DRAWINGS_POS_CURRENT_CNT_DRAWINGS_CURRENT_ratio', 'AMT_DRAWINGS_ATM_CURRENT_CNT_DRAWINGS_ATM_CURRENT_ratio', 'AMT_DRAWINGS_POS_CURRENT_CNT_DRAWINGS_POS_CURRENT_r

# Numerical Columns

In [15]:
numerical_cols = ['SK_DPD', 'SK_DPD_DEF','AMT_BALANCE','AMT_CREDIT_LIMIT_ACTUAL_minus_AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY', 'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM',  'AMT_DRAWINGS_ATM_CURRENT_AMT_DRAWINGS_CURRENT_ratio', 'AMT_DRAWINGS_POS_CURRENT_AMT_DRAWINGS_CURRENT_ratio', 'AMT_DRAWINGS_OTHER_CURRENT_AMT_DRAWINGS_CURRENT_ratio', 'CNT_DRAWINGS_ATM_CURRENT_CNT_DRAWINGS_CURRENT_ratio', 'CNT_DRAWINGS_OTHER_CURRENT_CNT_DRAWINGS_CURRENT_ratio', 'CNT_DRAWINGS_POS_CURRENT_CNT_DRAWINGS_CURRENT_ratio', 'AMT_DRAWINGS_ATM_CURRENT_CNT_DRAWINGS_ATM_CURRENT_ratio', 'AMT_DRAWINGS_POS_CURRENT_CNT_DRAWINGS_POS_CURRENT_ratio', 'AMT_DRAWINGS_OTHER_CURRENT_CNT_DRAWINGS_OTHER_CURRENT_ratio', 'AMT_DRAWINGS_CURRENT_CNT_DRAWINGS_CURRENT_ratio','score_DPD']

In [16]:
numerical_len = len(numerical_cols)
extrafunc_list = [_ for _ in range(len(numerical_cols))]
for i in range(len(numerical_cols)):
    if i == 0 or i == 1:
        extrafunc_list[i] = ['positive_count']
    else:
        if i == 2:
            extrafunc_list[i] = ['positive_count', 'negative_count']
        else:
            extrafunc_list[i] = None
print(extrafunc_list)

[['positive_count'], ['positive_count'], ['positive_count', 'negative_count'], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [17]:
def numerical_agg(df, gp_col, agg_col, df_name, extrafunc_list = None):
    
    agg_list = ['max', 'min', 'std','mean']
    '''
    Possible extrafunc_list: ['sum','median','two_minus_one_third','positive_count', 'negative_count','standard_error', 'trimmed_mean_10_pct', 'trimmed_mean_25_pct','normed_std', 'max_minus_min','one_third','two_third']
    '''
    if extrafunc_list:
        if 'one_third' in extrafunc_list:
            def one_third(series):
                return series.quantile(1/3)
            
        if 'two_third' in extrafunc_list:
            def two_third(series):
                return series.quantile(2/3)
            
        if 'max_minus_min' in extrafunc_list:
            def max_minus_min(series):
                return (series.max() - series.min())
        
        if 'two_minus_one_third' in extrafunc_list:
#             print('two_minus_one')
#             print('get two minus one')
            def two_minus_one_third(series):
                return(series.quantile(0.66666) - series.quantile(0.33333))
       
        if 'positive_count' in extrafunc_list:
#             print('get positive count')
            def positive_count(series):
                return pd.Series(series > 0).sum()

        if 'negative_count' in extrafunc_list:
#             print('get negative count')
            def negative_count(series):
                return pd.Series(series < 0).sum()

        if 'standard_error' in extrafunc_list:
#             print('get std-error')
            def standard_error(series):
                return series.std()/np.sqrt(len(series))

        if 'normed_std' in extrafunc_list:
#             print('get normed_std')
            def normed_std(series):
                return series.std()/series.mean()

        if 'trimmed_mean_10_pct' in extrafunc_list:
#             print('get_trimmed 10 pct')
            def trimmed_mean_10_pct(series):
                return stats.trim_mean(series.dropna(), 0.1)

        if 'trimmed_mean_25_pct' in extrafunc_list:
#             print('get_trimmed 25 pct')
            def trimmed_mean_25_pct(series):
                return stats.trim_mean(series.dropna(), 0.25)
        
        list_tocall = []
        for i in extrafunc_list:
            if i not in set(['sum', 'median']):
                list_tocall.append(eval(i))
            
    if extrafunc_list != None:   
        agg_list.extend(list_tocall)
    
    _df = df.groupby(gp_col).agg({agg_col:agg_list})
    columns = []
    for pre in _df.columns.levels[0]:
        for middle in _df.columns.levels[1]:
            columns.append('%s_%s_%s' %(df_name,pre,middle))
    _df.columns = columns
    
    return _df.reset_index()

In [18]:
gp_col = 'SK_ID_CURR'
for i in range(len(extrafunc_list)):
    print(numerical_cols[i])
    train = train.merge(numerical_agg(credit_card_balance, gp_col, numerical_cols[i], 'credit_card', extrafunc_list = extrafunc_list[i]), on = 'SK_ID_CURR', how='left')

SK_DPD
SK_DPD_DEF
AMT_BALANCE
AMT_CREDIT_LIMIT_ACTUAL_minus_AMT_BALANCE
AMT_CREDIT_LIMIT_ACTUAL
AMT_DRAWINGS_ATM_CURRENT
AMT_DRAWINGS_CURRENT
AMT_DRAWINGS_OTHER_CURRENT
AMT_DRAWINGS_POS_CURRENT
AMT_INST_MIN_REGULARITY
AMT_PAYMENT_CURRENT
AMT_PAYMENT_TOTAL_CURRENT
AMT_RECEIVABLE_PRINCIPAL
AMT_RECIVABLE
AMT_TOTAL_RECEIVABLE
CNT_DRAWINGS_ATM_CURRENT
CNT_DRAWINGS_CURRENT
CNT_DRAWINGS_OTHER_CURRENT
CNT_DRAWINGS_POS_CURRENT
CNT_INSTALMENT_MATURE_CUM
AMT_DRAWINGS_ATM_CURRENT_AMT_DRAWINGS_CURRENT_ratio
AMT_DRAWINGS_POS_CURRENT_AMT_DRAWINGS_CURRENT_ratio
AMT_DRAWINGS_OTHER_CURRENT_AMT_DRAWINGS_CURRENT_ratio
CNT_DRAWINGS_ATM_CURRENT_CNT_DRAWINGS_CURRENT_ratio
CNT_DRAWINGS_OTHER_CURRENT_CNT_DRAWINGS_CURRENT_ratio
CNT_DRAWINGS_POS_CURRENT_CNT_DRAWINGS_CURRENT_ratio
AMT_DRAWINGS_ATM_CURRENT_CNT_DRAWINGS_ATM_CURRENT_ratio
AMT_DRAWINGS_POS_CURRENT_CNT_DRAWINGS_POS_CURRENT_ratio
AMT_DRAWINGS_OTHER_CURRENT_CNT_DRAWINGS_OTHER_CURRENT_ratio
AMT_DRAWINGS_CURRENT_CNT_DRAWINGS_CURRENT_ratio
score_DPD


KeyError: 'score_DPD'

In [21]:
credit_card_balance['score_DPD']

24268      0.010309
841327     0.010309
2725728    0.010309
2725726    0.010309
841332     0.010309
2725714    0.010309
2725711    0.010309
841338     0.072165
2725708    0.010309
841340     0.010309
2725701    0.010309
2725698    0.010309
2725693    0.010309
841347     0.010309
2725682    0.010309
841355     0.010309
1326641    0.010309
2725674    0.010309
841362     0.010309
2725667    0.010309
844154     0.010309
844163     0.010309
844164     0.010309
2465407    0.010309
2725654    0.010309
2725637    0.010309
3369124    0.010309
845859     0.010309
841322     0.072165
2725737    0.010309
             ...   
3049386    0.500000
3049387    0.500000
1478565    0.500000
395943     0.500000
395942     0.500000
395939     0.500000
395869     0.500000
395870     0.500000
395871     0.500000
395874     0.500000
395876     0.500000
395878     0.500000
395881     0.500000
395884     0.500000
395885     0.500000
395887     0.500000
395866     0.500000
395891     0.500000
395905     0.500000


# Categorical

In [22]:
def _set_type(series, dtype):
    _max, _min = max(series), min(series)
    if dtype == 'uint':
        if _max <= 255: return np.uint8
        elif _max <= 65535: return np.uint16
        elif _max <= 4294967295: return np.uint32
        else: return np.uint64
    elif dtype == 'int':
        if _min >= -128 and _max <= 127: return np.int8
        elif _min >=-32768 and _max <= 32767: return np.int16
        elif _min >= -2147483648 and _max <= 2147483647: return np.int32
        else: return np.int64
    elif dtype == 'float':
        if max(abs(_min), _max) <= 3.4028235e+38: return np.float32
        else: return np.float64

def split_categorical_feature(df, group_col, calc_col):
    tmp_df = pd.concat([df[group_col], pd.get_dummies(df[calc_col], prefix=calc_col)], axis=1).groupby(by=group_col).sum().reset_index()
    dtype = {x: _set_type(tmp_df[x], 'uint') for x in tmp_df.columns if x != group_col}
    for x in tmp_df:
        if x in df.columns:
            dtype[x] = df[x].dtype
    return tmp_df.astype(dtype)

In [23]:
categorical_cols = ['NAME_CONTRACT_STATUS']

for x in categorical_cols:
    print(x)
    train = train.merge(split_categorical_feature(credit_card_balance, 'SK_ID_CURR', x), on='SK_ID_CURR', how='left')
    gc.collect()

NAME_CONTRACT_STATUS
